In [1]:
import logging
import sqlite3
from pathlib import Path

import kagglehub
import pandas as pd
from kagglehub.exceptions import KaggleApiHTTPError

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

In [2]:
try:
    csv_path = kagglehub.dataset_download(handle="urvishahir/electric-vehicle-specifications-dataset-2025")
    logging.info(f"Dataset downloaded to: {csv_path}")
except KaggleApiHTTPError as e:
    logging.error(f"Error downloading dataset: {e}")
    raise

csv_files = Path(csv_path).glob("*.csv")
csv_file = next(csv_files, None)

2025-07-14 20:45:16,528 - DEBUG - Starting new HTTPS connection (1): www.kaggle.com:443
2025-07-14 20:45:16,854 - DEBUG - https://www.kaggle.com:443 "GET /api/v1/datasets/view/urvishahir/electric-vehicle-specifications-dataset-2025 HTTP/1.1" 200 None
2025-07-14 20:45:16,919 - DEBUG - Starting new HTTPS connection (1): www.kaggle.com:443
2025-07-14 20:45:17,186 - DEBUG - https://www.kaggle.com:443 "GET /api/v1/datasets/download/urvishahir/electric-vehicle-specifications-dataset-2025?dataset_version_number=1 HTTP/1.1" 302 0
2025-07-14 20:45:17,189 - DEBUG - Starting new HTTPS connection (1): storage.googleapis.com:443
2025-07-14 20:45:17,455 - DEBUG - https://storage.googleapis.com:443 "GET /kaggle-data-sets/7702238/12225161/bundle/archive.zip?X-Goog-Algorithm=GOOG4-RSA-SHA256&X-Goog-Credential=gcp-kaggle-com%40kaggle-161607.iam.gserviceaccount.com%2F20250714%2Fauto%2Fstorage%2Fgoog4_request&X-Goog-Date=20250714T204517Z&X-Goog-Expires=259200&X-Goog-SignedHeaders=host&X-Goog-Signature=18f

100%|██████████| 16.2k/16.2k [00:00<00:00, 26.8MB/s]

Extracting files...



2025-07-14 20:45:17,491 - INFO - Dataset downloaded to: /home/vscode/.cache/kagglehub/datasets/urvishahir/electric-vehicle-specifications-dataset-2025/versions/1


In [3]:
db_file = Path("ev_specifications.db")

conn = sqlite3.connect(db_file)

df = pd.read_csv(csv_file)
rows = df.to_sql("ev_specifications", conn, if_exists="replace", index=False)

logging.info(f"Number of rows inserted: {rows}")

conn.close()

2025-07-14 20:45:17,524 - INFO - Number of rows inserted: 478


In [4]:
conn = sqlite3.connect(db_file)
brand = "Hyundai"
query = "SELECT * FROM ev_specifications WHERE brand = :brand"

df_hyundai = pd.read_sql_query(query, conn, params={"brand": brand})
print("Number of HYUNDAI cars:", len(df_hyundai))
print(df_hyundai.head())

Number of HYUNDAI cars: 14
     brand                      model  top_speed_kmh  battery_capacity_kWh  \
0  Hyundai          INSTER Long Range            150                  46.0   
1  Hyundai      INSTER Standard Range            140                  39.0   
2  Hyundai  IONIQ 5 63 kWh RWD (MY24)            185                  60.0   
3  Hyundai  IONIQ 5 84 kWh AWD (MY24)            185                  80.0   
4  Hyundai  IONIQ 5 84 kWh RWD (MY24)            185                  80.0   

  battery_type  number_of_cells  torque_nm  efficiency_wh_per_km  range_km  \
0  Lithium-ion              NaN      147.0                   124       300   
1  Lithium-ion              NaN      147.0                   119       255   
2  Lithium-ion            288.0      350.0                   136       345   
3  Lithium-ion            384.0      605.0                   162       445   
4  Lithium-ion            384.0      350.0                   155       450   

   acceleration_0_100_s  ...  towin